# Imports y configuraciones iniciales

In [ ]:
import warnings

from typing import List

warnings.filterwarnings("ignore")

In [ ]:
# Dejar el path principal como el anterior.
import sys

sys.path.append('../')

In [ ]:
import logging

# Crear el logger
log = logging.getLogger(__name__)

# Setear el nivel del registro
log.setLevel(logging.WARNING)

# Formato de los mensajes
formatter = logging.Formatter("%(levelname)s: (%(asctime)s) [%(filename)s: %(lineno)s] %(message)s")

if not log.hasHandlers():
    # Handlers
    file_handler = logging.FileHandler("logging.log")
    file_handler.setFormatter(formatter)  # Setear el formato del handler
    # Agregar el handler al logger
    # log.addHandler(file_handler)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    # Agregar el handler al logger
    log.addHandler(stream_handler)

Instalar la librería plot-likert y otras librerías útiles

In [ ]:
# Librería para hacer gráficos Likert
# !pip install plot-likert

# Para obtener datos de excel
# !pip install openpyxl

# Para tener un transformador de data
# !pip install -U scikit-learn

# Para tener herramientas estadísticas
# !python -m pip install statsmodels

# Para tener Seaborn
# !pip install seaborn

Importamos algunas librerías útiles para el resto del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Importación del data set

Empezamos obteniendo los datos

In [ ]:
DATA_PATH = Path(r"..\data")

if not DATA_PATH.exists():
    log.info(f"Creando carpeta {DATA_PATH = }")
    DATA_PATH.mkdir()

In [ ]:
QNA_COMPLETE_PATH = DATA_PATH / "questions-and-answers-complete.xlsx"
QNA_COMPLETE_PATH

In [ ]:
log.debug(f"Importando datos de {QNA_COMPLETE_PATH}")
df = pd.read_excel(QNA_COMPLETE_PATH)

log.debug(f"Cantidad de respuestas: {len(df) = }")

columnas = list(df.columns)
preguntas = columnas[1:]
notas = columnas[0]

df.head()

In [ ]:
questions_ = preguntas.copy()
# Separar por subdimensiones

# Subdim 1.1
subdim_1_1 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.2
subdim_1_2 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.3
subdim_1_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0)]

# Subdim 2.1
subdim_2_1 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.2
subdim_2_2 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.3
subdim_2_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 2.4
subdim_2_4 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0), questions_.pop(0)]

del questions_

In [ ]:
questions = list(df.columns)[1:]
questions

In [ ]:
import copy


log.debug("Renombrando columnas")
questions_copy = copy.copy(questions)
for i in range(len(questions)):
    # questions_copy[i] = f"P{i+1}.: " + questions[i]
    questions_copy[i] = f"P{i+1}"

questions = questions_copy.copy()

df.columns = ["Notas"] +  questions_copy


df.head()

In [ ]:
scales = [
    "Nunca",
    "Rara vez",
    "Ocasionalmente",
    "Casi siempre",
    "Siempre, en todas las clases"
]

In [ ]:
df

Codificamos las preguntas para calcular el VIF

In [ ]:
from collections import OrderedDict
df_enc = OrderedDict()

encoder = {name: i-2 for i, name in enumerate(scales)}
for col_name in questions_copy:
    df_enc[col_name] = df[col_name].map(encoder)

df_enc["Notas"] = (df["Notas"] - 1) / 6

df_enc = pd.DataFrame(df_enc)
df_enc

Asignamos a la variable `X` las columnas a determinar si poseen algún tipo de correlación

In [ ]:
X = df_enc[questions]
X

Armamos la matriz con los valores del VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

vif_data

Obtenemos aquellas preguntas que servirán para hacer la regresión

In [ ]:
vif_data[vif_data["VIF"] <= 5]

In [ ]:
vif_data[vif_data["VIF"] <= 12]